<a href="https://www.kaggle.com/code/umairalam567/license?scriptVersionId=144474407" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import os
import shutil
import os

shutil.copytree('/kaggle/input/car-plate-detection', '/kaggle/working/car-plate-detection')

## Code Explanation

The code is designed to convert XML annotations (typically used in object detection tasks) to the YOLO (You Only Look Once) format. The YOLO format is a popular format for object detection tasks, especially when using the YOLO architecture.

### Importing Necessary Libraries
```python
import os
import xml.etree.ElementTree as ET
import shutil
```

- `os`: This module provides a way of using operating system dependent functionality.
- `xml.etree.ElementTree`: This module is used for parsing XML data.
- `shutil`: This module helps in automating the process of copying and removal of files and directories.

### Function: `convert_annotation`
This function converts a single XML annotation file to the YOLO format.

- It first parses the XML file to get the root of the XML tree.
- Extracts the image dimensions (width and height).
- Iterates over each object in the XML (in this case, looking for objects named 'licence').
- For each 'licence' object, it extracts the bounding box coordinates.
- Converts these coordinates to the YOLO format, which is normalized and relative to the image dimensions.
- Writes the converted annotation to an output file.

### Function: `main`
This is the main driver function.

- It first defines the directories for images, XML annotations, and the output directories for training and validation splits.
- It ensures that the output directories exist, and if not, creates them.
- It then lists all XML files in the annotation directory.
- For each XML file, it determines the corresponding image file.
- Based on a predefined split (80% training, 20% validation), it copies the image to the respective directory (training or validation).
- Calls the `convert_annotation` function to convert the XML annotation to YOLO format and saves it in the respective directory.

Finally, the `main` function is called to execute the entire process.

### Note:
The code assumes that for each XML file, there is a corresponding image with the same name but with a '.png' extension. It also assumes that the only object of interest in the XML files is 'licence', and it assigns this object a class ID of 0 in the YOLO format.

In [ ]:
import os
import xml.etree.ElementTree as ET
import shutil

def convert_annotation(xml_path, img_path, output_label_path):
    try:
        tree = ET.parse(xml_path)
        root = tree.getroot()

        # Extract image dimensions
        size = root.find('size')
        w = int(size.find('width').text)
        h = int(size.find('height').text)

        # Open output file
        with open(output_label_path, 'w') as out_file:
            for obj in root.iter('object'):
                # Get class name (assuming 'licence' is class 0)
                cls_name = obj.find('name').text
                if cls_name == 'licence':
                    cls_id = 0
                else:
                    continue

                # Get bounding box coordinates
                xmlbox = obj.find('bndbox')
                b = (float(xmlbox.find('xmin').text), float(xmlbox.find('ymin').text),
                     float(xmlbox.find('xmax').text), float(xmlbox.find('ymax').text))
                bb = (b[0] + b[2]) / 2 / w, (b[1] + b[3]) / 2 / h, (b[2] - b[0]) / w, (b[3] - b[1]) / h

                # Write to output file in YOLO format
                out_file.write(str(cls_id) + ' ' + ' '.join([str(a) for a in bb]) + '\n')

    except Exception as e:
        print(f'Error processing {xml_path}: {e}')

def main():
    # Directories
    base_dir = '/kaggle/working/car-plate-detection'
    img_dir = os.path.join(base_dir, 'images')
    xml_dir = os.path.join(base_dir, 'annotations')
    
    train_img_output_dir = os.path.join(base_dir, 'train/images')
    train_label_output_dir = os.path.join(base_dir, 'train/labels')
    val_img_output_dir = os.path.join(base_dir, 'val/images')
    val_label_output_dir = os.path.join(base_dir, 'val/labels')

    # Create output directories if they don't exist
    for dir_path in [train_img_output_dir, train_label_output_dir, val_img_output_dir, val_label_output_dir]:
        if not os.path.exists(dir_path):
            os.makedirs(dir_path)

    # Process each XML file
    xml_files = [f for f in os.listdir(xml_dir) if f.endswith('.xml')]
    total_files = len(xml_files)
    train_split = int(0.8 * total_files)  # 80% for training

    for idx, xml_file in enumerate(xml_files):
        img_file = xml_file.replace('.xml', '.png')
        if os.path.exists(os.path.join(img_dir, img_file)):
            if idx < train_split:
                img_output_path = os.path.join(train_img_output_dir, img_file)
                label_output_path = os.path.join(train_label_output_dir, img_file.replace('.png', '.txt'))
            else:
                img_output_path = os.path.join(val_img_output_dir, img_file)
                label_output_path = os.path.join(val_label_output_dir, img_file.replace('.png', '.txt'))
            
            # Copy image to respective directory
            shutil.copy(os.path.join(img_dir, img_file), img_output_path)
            
            # Convert annotation
            convert_annotation(os.path.join(xml_dir, xml_file), img_output_path, label_output_path)


main()


In [ ]:
cd /kaggle/working/car-plate-detection

In [ ]:
ls

## YOLO Model Loading Explanation

The provided code snippet demonstrates how to load a YOLO model using the `ultralytics` library. The `ultralytics` library provides utilities and functionalities for working with the YOLO (You Only Look Once) object detection architecture.

---

### **1. Importing the YOLO Class**

```python
from ultralytics import YOLO
```

This line imports the `YOLO` class from the `ultralytics` library, which is used for creating and managing YOLO models.

---

### **2. Loading a YOLO Model**

There are multiple ways to load a YOLO model, and the code snippet provides three examples:

1. **Building a New Model from YAML:**

```python
#model = YOLO('yolov8n.yaml')
```

This line (currently commented out) demonstrates how to build a new YOLO model using a YAML configuration file (`yolov8n.yaml`). The YAML file contains the architecture and configuration details of the model.

2. **Loading a Pretrained Model:**

```python
model = YOLO('yolov8s.pt')
```

This line loads a pretrained YOLO model from a `.pt` file (`yolov8s.pt`). This method is recommended when you want to continue training a model or use it for inference without starting from scratch.

3. **Building from YAML and Transferring Weights:**

```python
#model = YOLO('yolov8n.yaml').load('yolov8n.pt')
```

This line (currently commented out) first builds a YOLO model using a YAML configuration file (`yolov8n.yaml`) and then transfers the weights from a pretrained model (`yolov8n.pt`). This approach is useful when you want to use a specific architecture but initialize it with weights from a pretrained model.

---

### **Note**

In the provided code, only the second method (loading a pretrained model) is active, while the other two methods are commented out. Depending on the use case, you can choose the appropriate method to load the YOLO model.

In [ ]:
!pip install ultralytics

In [ ]:
from ultralytics import YOLO

# Load a model
#model = YOLO('yolov8n.yaml')  # build a new model from YAML
model = YOLO('yolov8s.pt')  # load a pretrained model (recommended for training)
#model = YOLO('yolov8n.yaml').load('yolov8n.pt')  # build from YAML and transfer weights

## YAML Configuration File Creation Explanation

The provided code snippet demonstrates how to create and save a configuration file in YAML format using the `yaml` library in Python.

---

### **1. Importing the YAML Library**

```python
import yaml
```

This line imports the `yaml` library, which provides functionalities to work with YAML (Yet Another Markup Language) files in Python.

---

### **2. Defining Configuration Data**

```python
config = {
    'path': '/kaggle/working/car-plate-detection',
    'train': 'train/images',
    'val': 'val/images',
    'nc': 1,
    'names': ['license'],
    # 'fl_gamma': 2.0
}
```

Here, a Python dictionary named `config` is defined to hold the configuration data. This data includes paths, number of classes, class names, and other potential configurations (like `fl_gamma`, which is currently commented out).

---

### **3. Specifying the File Path**

```python
yaml_file_path = '/kaggle/working/config.yaml'
```

This line defines the file path where the YAML configuration file will be saved.

---

### **4. Writing Data to the YAML File**

```python
with open(yaml_file_path, 'w') as yaml_file:
    yaml.dump(config, yaml_file)
```

Using the `with` statement, the code opens the specified file in write mode. The `yaml.dump()` function is then used to write the `config` dictionary data to the file in YAML format.

---

### **5. Printing the Save Location**

```python
print(f"YAML configuration file saved at: {yaml_file_path}")
```

This line simply prints the location where the YAML configuration file has been saved.

---

### **Note**

YAML is a human-readable data serialization format. It's commonly used for configuration files and data exchange between languages with different data structures. The provided code snippet is a simple example of how to create and save a YAML configuration file in Python.

In [ ]:
import yaml

# Define your configuration data as a Python dictionary
config = {
    'path': '/kaggle/working/car-plate-detection',
    'train': 'train/images',  # train images (relative to 'path') 4 images
    'val': 'val/images',      # val images (relative to 'path') 4 images
    'nc': 1,                   # Number of classes
    'names': ['license'],         # Class names
    # 'fl_gamma': 2.0          # Uncomment this line if needed
}

# Define the file path where you want to save the YAML file
yaml_file_path = '/kaggle/working/config.yaml'  # Update with your desired path and file name

# Write the YAML data to the file
with open(yaml_file_path, 'w') as yaml_file:
    yaml.dump(config, yaml_file)

print(f"YAML configuration file saved at: {yaml_file_path}")

## Model Training Explanation

The provided code snippet demonstrates how to train a YOLO model using the `train` method of the `YOLO` class from the `ultralytics` library.

---

### **Training the YOLO Model**

```python
results = model.train(data='/kaggle/working/config.yaml',
                      epochs=220,
                      imgsz=640,
                      pretrained=True,
                      name="license_plate_small3",
                      patience=35,
                      flipud=0.5,
                      batch=16,
                      optimizer='SGD',
                      lr0=0.001,
                      augment=True)
```

Here's a breakdown of the parameters used in the `train` method:

- **data**: Path to the YAML configuration file that contains dataset information.

- **epochs**: Number of training epochs. An epoch is one complete forward and backward pass of all the training examples.

- **imgsz**: Image size. The model will resize training images to this size.

- **pretrained**: If set to `True`, the model will use pretrained weights. This can help in achieving better accuracy faster.

- **name**: Name of the training run. Useful for distinguishing between different training sessions.

- **patience**: Number of epochs to wait for improvement before stopping training. Helps in early stopping if the model isn't improving.

- **flipud**: Probability of flipping an image upside down during training. Data augmentation technique to improve model robustness.

- **batch**: Batch size for training. Number of training examples utilized in one iteration.

- **optimizer**: The optimization algorithm to use. In this case, Stochastic Gradient Descent (SGD) is used.

- **lr0**: Initial learning rate. Determines the step size at each iteration while moving towards a minimum of the loss function.

- **augment**: If set to `True`, data augmentation techniques will be applied during training. This can help in improving model accuracy.

---

### **Note**

The `train` method returns a `results` object which contains information about the training process, such as loss values, accuracy metrics, etc. This can be used later for analysis and visualization.

In [ ]:
results = model.train(data='/kaggle/working/config.yaml',
                      epochs=220,
                      imgsz=640,
                      pretrained = True ,
                      name= "license_plate_small3",
                      patience = 35,
                      flipud=0.5,
                      batch = 16,
                      optimizer = 'SGD',
                      lr0 = 0.001,
                      augment = True
                      )

In [ ]:
!cat /kaggle/working/car-plate-detection/val/labels/Cars4.txt

## Model Validation Explanation

The provided code snippet demonstrates how to validate a YOLO model using the `val` method of the `YOLO` class from the `ultralytics` library.

---

### **1. Loading the YOLO Model**

```python
model = YOLO('/kaggle/working/car-plate-detection/runs/detect/license_plate_small3/weights/best.pt')
```

This line loads a pretrained YOLO model from a `.pt` file. The path provided points to the best weights obtained during a previous training session.

---

### **2. Default Model Validation**

```python
results = model.val()
```

This line validates the model using default parameters. The `val` method returns a `results` object containing information about the validation process, such as precision, recall, and F1 score.

---

### **3. Model Validation with Different Confidence Thresholds**

```python
for i in [0.25, 0.15, 0.05]:
    results = model.val(name=f'confidence: {i}', conf=i, iou=0.8)
```

This loop validates the model three times, each with a different confidence threshold (`conf`). The confidence threshold determines which detections are considered by the model. Detections with a confidence score below this threshold are discarded.

- **name**: A custom name for the validation run. Useful for distinguishing between different validation sessions.

- **conf**: The confidence threshold for detections.

- **iou**: Intersection over Union threshold. Determines how much overlap there should be between the predicted bounding box and the ground truth bounding box for a detection to be considered correct.

In this case, the model is validated with confidence thresholds of 0.25, 0.15, and 0.05, while keeping the IoU threshold constant at 0.8.

---

### **Note**

Validating the model with different confidence thresholds can help in understanding how the model performs under different conditions. It can provide insights into the optimal confidence threshold to use for the specific application.

In [ ]:
model = YOLO('/kaggle/working/car-plate-detection/runs/detect/license_plate_small3/weights/best.pt')
results = model.val()
for i in [0.25,0.15,0.05]:
  results = model.val(name= f'cofidence: {i}', conf= i , iou=0.8)

In [ ]:
model = YOLO('/kaggle/working/car-plate-detection/runs/detect/license_plate_small3/weights/best.pt')


## Python Libraries Import Explanation

The provided code snippet imports various Python libraries and modules that are commonly used in data analysis, image processing, and visualization tasks.

---

### **1. Data Analysis Libraries**

```python
import pandas as pd
import numpy as np
```

- `pandas`: A powerful library for data manipulation and analysis. It provides data structures like DataFrame for handling and analyzing structured data.

- `numpy`: A library for numerical computing in Python. It provides support for arrays (including multidimensional arrays), as well as a collection of mathematical functions to operate on these arrays.

---

### **2. Image Processing and Visualization Libraries**

```python
import PIL
from PIL import Image
from IPython.display import display
import matplotlib.pyplot as plt
import cv2
```

- `PIL` and `Image`: Modules from the Python Imaging Library (PIL), which provides capabilities to open, manipulate, and save various image file formats.

- `display`: A function from IPython's display module to display PIL images, DataFrames, and other objects in Jupyter notebooks.

- `matplotlib.pyplot`: A module in the Matplotlib library for plotting and visualization in Python.

- `cv2`: OpenCV library, which provides tools for image and video analysis, including object detection, image segmentation, and more.

---

### **3. File Handling and Miscellaneous**

```python
from glob import glob
import random
import warnings
warnings.simplefilter('ignore')
```

- `glob`: A function to search for files based on patterns and wildcards.

- `random`: A module that implements pseudo-random number generators for various distributions.

- `warnings`: A module to handle warnings. The `simplefilter('ignore')` line is used to suppress warnings, ensuring they don't clutter the output.

---

### **Note**

It's a good practice to import all necessary libraries at the beginning of a script or notebook. This ensures that all dependencies are available before executing subsequent code.

In [ ]:
import pandas as pd
import numpy as np
import PIL 
from PIL import Image
from IPython.display import display
import matplotlib.pyplot as plt
from glob import glob
import random
import cv2
import warnings
warnings.simplefilter('ignore')

## Random Image Display Explanation

The provided code snippet demonstrates how to randomly select and display images from a specified directory using Python libraries.

---

### **1. Setting the Image Directory and Number of Samples**

```python
root_path = '/kaggle/working/car-plate-detection/val/images/*'
num_samples = 4
```

- `root_path`: Specifies the directory path where the images are stored. The wildcard `*` at the end means it will consider all files in that directory.

- `num_samples`: Specifies the number of random images to display.

---

### **2. Fetching Image Paths and Randomly Selecting Images**

```python
images_data = glob(root_path)
random_image = random.sample(images_data, num_samples)
```

- `glob(root_path)`: Fetches the paths of all files in the specified directory.

- `random.sample()`: Randomly selects a specified number of items from a list. In this case, it selects `num_samples` image paths from `images_data`.

---

### **3. Displaying the Randomly Selected Images**

```python
plt.figure(figsize=(12,10))
for i in range(num_samples):
    plt.subplot(2,2,i+1)
    plt.imshow(cv2.imread(random_image[i]))
    plt.axis('off')
```

- `plt.figure()`: Initializes a new figure for plotting. The `figsize` parameter sets the width and height of the figure in inches.

- `plt.subplot()`: Divides the figure into a grid (in this case, 2x2) and selects a specific subplot to plot on. The loop ensures that each image is plotted on a separate subplot.

- `cv2.imread()`: Reads an image from the specified path.

- `plt.imshow()`: Displays the image on the current subplot.

- `plt.axis('off')`: Hides the axis labels and ticks for a cleaner display.

---

### **Note**

This code snippet is useful for quickly visualizing a random subset of images from a dataset, which can be helpful in understanding the nature and quality of the data.

In [ ]:
root_path = '/kaggle/working/car-plate-detection/val/images/*'
num_samples = 4
images_data = glob(root_path)
random_image = random.sample(images_data, num_samples)

plt.figure(figsize=(12,10))
for i in range(num_samples):
    plt.subplot(2,2,i+1)
    plt.imshow(cv2.imread(random_image[i]))
    plt.axis('off')

## YOLO Model Prediction and Visualization Explanation

The provided code snippet demonstrates how to use a YOLO model to predict and visualize object detections on a set of randomly selected images.

---

### **1. Initializing an Empty List for Images**

```python
images = []
```

This line initializes an empty list named `images` which will be used to store the visualized output images with detected objects.

---

### **2. Looping Through the Randomly Selected Images**

```python
for i in range(num_samples):
```

This loop iterates through each of the randomly selected images.

---

### **3. Predicting Objects in the Image**

```python
yolo_outputs = model.predict(random_image[i])
output = yolo_outputs[0]
```

- `model.predict()`: Uses the YOLO model to predict objects in the given image.

- `output`: Extracts the first output from the prediction, which contains information about the detected objects.

---

### **4. Extracting Detection Information**

```python
box = output.boxes
names = output.names
```

- `box`: Contains bounding box information of the detected objects.

- `names`: Contains the names (or labels) of the detected objects.

---

### **5. Displaying Detection Details**

The inner loop iterates through each detected object and prints details such as the label, coordinates of the bounding box, and the confidence score.

---

### **6. Storing the Visualized Image**

```python
images.append(output.plot()[:, :, ::-1])
```

- `output.plot()`: Visualizes the detected objects on the image.

- `[:, :, ::-1]`: Converts the image from BGR to RGB format (as OpenCV reads images in BGR format by default).

- `images.append()`: Appends the visualized image to the `images` list.

---

### **Note**

This code snippet is useful for visualizing the performance of the YOLO model on a random subset of images. It provides insights into the detected objects, their locations, and the model's confidence in each detection.

In [ ]:
images = []
for i in range(num_samples):
    yolo_outputs = model.predict(random_image[i])
    output = yolo_outputs[0]
    box = output.boxes
    names = output.names
    print('**********************')
    for j in range(len(box)):
        labels = names[box.cls[j].item()]
        coordinates = box.xyxy[j].tolist()
        confidence = np.round(box.conf[j].item(), 2)
        print(f'In this image {len(box)} License plate has been detected.')
        print(f'License {j + 1} is: {labels}')
        print(f'Coordinates are: {coordinates}')
        print(f'Confidence is: {confidence}')
        print('-------')
        
    # Store the image in the 'images' list
    images.append(output.plot()[:, :, ::-1])

In [ ]:
plt.figure(figsize=(12,10))
for i, img in enumerate(images):
    plt.subplot(2, 2, i + 1)
    plt.imshow(img)
    plt.axis('off') 

In [ ]:
result = pd.read_csv('/kaggle/working/car-plate-detection/runs/detect/license_plate_small3/results.csv')
result.head()

In [ ]:
# Remove leading and trailing spaces from column names
result.columns = result.columns.str.strip()

epoch_column = result['epoch']
box_train_losses = result['train/box_loss']
box_val_losses = result['val/box_loss']
cls_train_losses = result['train/cls_loss']
cls_val_losses = result['val/cls_loss']

plt.figure(figsize=(12,5))
plt.style.use('ggplot')  # You can choose a style you prefer
plt.subplot(1,2,1)
plt.plot(epoch_column, box_train_losses, label='train_losses')
plt.plot(epoch_column, box_val_losses, label='val_losses')
plt.grid(True, linestyle='--', linewidth=0.5, color='gray')# Add a grid
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Train and Validation Box Losses')
plt.legend()

plt.subplot(1,2,2)
plt.plot(epoch_column, cls_train_losses, label='train_losses')
plt.plot(epoch_column, cls_val_losses, label='val_losses')
plt.grid(True, linestyle='--', linewidth=0.5, color='gray')# Add a grid
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Train and Validation Class Losses')
plt.legend()
plt.show()

In [ ]:
plt.figure(figsize=(12,8))
plt.imshow(cv2.imread('/kaggle/working/car-plate-detection/runs/detect/license_plate_small3/results.png'))
plt.axis('off')

## Image Text Extraction and Visualization Explanation

The provided code snippet demonstrates how to use a combination of YOLO for object detection and Tesseract for Optical Character Recognition (OCR) to extract text from detected objects in images.

---

### **1. Importing Necessary Libraries**

The code starts by importing necessary libraries such as `cv2`, `pytesseract`, `PIL`, `numpy`, `matplotlib`, `glob`, and `random`.

---

### **2. Defining the Text Extraction Function**

```python
def extract_text_from_image(image, coordinates):
    ...
```

This function extracts text from a specified region of an image using the Tesseract OCR engine. The region is defined by the provided bounding box `coordinates`.

---

### **3. Setting the Image Directory and Number of Samples**

```python
root_path = '/kaggle/working/car-plate-detection/val/images/*'
num_samples = 2
```

This sets the directory path for the images and the number of random images to process.

---

### **4. Randomly Selecting Images**

```python
images_data = glob(root_path)
random_image_paths = random.sample(images_data, num_samples)
```

This code fetches all image paths from the specified directory and then randomly selects a subset of them.

---

### **5. Processing and Displaying the Images**

The main loop processes each randomly selected image. For each image:

- The YOLO model predicts objects.

- Detected license plates' details (label, coordinates, confidence) are printed.

- The `extract_text_from_image` function is called to extract text from the detected license plate using OCR.

- The processed image is displayed using `matplotlib`.

---

### **Note**

This code snippet provides an end-to-end workflow for detecting license plates in images and then extracting text from them using OCR. It's a common approach for applications like automatic license plate recognition.

In [ ]:
import cv2
import pytesseract
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
from glob import glob
import random

def extract_text_from_image(image, coordinates):
    """
    Extract text from the given image using the provided coordinates.
    
    Args:
    - image (numpy array): The image from which to extract text.
    - coordinates (list): The bounding box coordinates [xmin, ymin, xmax, ymax].
    
    Returns:
    - str: The extracted text.
    """
    xmin, ymin, xmax, ymax = map(int, coordinates)
    roi = image[ymin:ymax, xmin:xmax]
    text = pytesseract.image_to_string(roi, config='--psm 8')
    return text.strip()

root_path = '/kaggle/working/car-plate-detection/val/images/*'
num_samples = 2
images_data = glob(root_path)
random_image_paths = random.sample(images_data, num_samples)

plt.figure(figsize=(12,10))
for i in range(num_samples):
    image_np = cv2.imread(random_image_paths[i])
    yolo_outputs = model.predict(image_np)
    output = yolo_outputs[0]
    box = output.boxes
    names = output.names
    print('**********************')
    for j in range(len(box)):
        labels = names[box.cls[j].item()]
        coordinates = box.xyxy[j].tolist()
        confidence = np.round(box.conf[j].item(), 2)
        print(f'In this image {len(box)} License plate has been detected.')
        print(f'License {j + 1} is: {labels}')
        print(f'Coordinates are: {coordinates}')
        print(f'Confidence is: {confidence}')
        
        # Extract text using OCR
        extracted_text = extract_text_from_image(image_np, coordinates)
        print(f'Extracted text from license plate: {extracted_text}')
        
        print('-------')
        
    # Display the image
    plt.subplot(2, 2, i + 1)
    plt.imshow(cv2.cvtColor(image_np, cv2.COLOR_BGR2RGB))  # Convert BGR to RGB for displaying with matplotlib
    plt.axis('off')
plt.show()